## Import necessary modules

In [ ]:
import cantera as ct
import numpy as np
import pylab as plt

## Specify input files and phases

In [ ]:
# File for gas phase model: GRI 3.0 (CH4 combution)
gasfile = 'gri30.cti'
gas_phase = 'gri30_mix'

## Create a gas phase Cantera object
Set T and P to 300 K, 1 atm:

In [ ]:
gas = ct.Solution(gasfile,gas_phase)
gas.TP = 300, ct.one_atm
gas()

## Here is some input/output for composition:
We can read out properties to variable arrays:

In [ ]:
molefracs = gas.X
print(molefracs)

## Set all molefractions to 1/n_species:

In [ ]:
nsp = gas.n_species
print(nsp)
gas.X = np.ones(molefracs.shape)/nsp
gas()

## Another, equivalent method:

In [ ]:
molefracs = np.ones(molefracs.shape)
gas.TPX = None, None, molefracs
gas()

## Thermodynamic Properties:

In [ ]:
mu_k = gas.chemical_potentials
print(mu_k)

## Transport Properties:

In [ ]:
D_jk = gas.binary_diff_coeffs
print(D_jk.shape)
D_jk

# Battery Simulation Capabilities

## List phase names:

In [ ]:
# File for battery anode simulation: developed in-house
batfile_1 = 'W_anode.cti'
# File with more complex SEI chemistry:
batfile_2 = 'W_anode_chem.cti'

anode_phase = 'tungsten'
elyte_phase = 'electrolyte'
SEI_phase = 'SEI'
anode_SEI_int = 'tungsten_SEI_surf'
SEI_elyte_int = 'SEI_electrolyte_surf'
anode_elyte_int = 'tungsten_electrolyte_surf'


## Initialize Cantera objects

In [ ]:
infile = batfile_1
anode = ct.Solution(infile,anode_phase)
elyte = ct.Solution(infile,elyte_phase)
SEI = ct.Solution(infile,SEI_phase)
anode_elyte = ct.Interface(infile,anode_elyte_int,[anode,elyte,SEI])

# Set the phase properties (including electric potentials):

In [ ]:
anode.TP = 300, ct.one_atm
elyte.TP = 300, ct.one_atm

anode.electric_potential = 1.2
anode_elyte.electric_potential = 1.2
elyte.electric_potential = 0.0
SEI.X = '(dummy):1.0, Li2CO3(SEI):1e-6'
anode()
elyte()

## Read out species production rates at anode/electrolyte interface:

Faradaic current:
\begin{equation}
i = F\dot{s}_{Li^+}
\end{equation}

Conservation of anode species:

\begin{equation}
\frac{\partial X_{k,\,{\rm graphite}}}{\partial t} = -\nabla J_k 
\end{equation}

where:
\begin{equation}
J_k|_{r=R} = \dot{s}_{k,\, int}
\end{equation}

and:

\begin{equation}
\dot{s}_{k,\,int} = \nu_{k,j}\dot{q}_j,
\end{equation}

\begin{equation}
\dot{q}_j = k_{fwd}\prod\left[C_{ac,k}\right]^{\nu_{k,j}^\prime}\exp\left(-\frac{nF\beta_{fwd}\Delta\Phi}{RT}\right) - k_{rev}\prod\left[C_{ac,k}\right]^{\nu_{k,j}^{\prime\prime}}\exp\left(\frac{nF\beta_{rev}\Delta\Phi}{RT}\right)
\end{equation}
and:
\begin{equation}
k_{rev} = \frac{k_{fwd}}{K_{\rm eq}}
\end{equation}

In [ ]:
sdot = anode_elyte.net_production_rates

Production rates are in the order (from the cti file):

<p1><center>Tungsten Species (1)</center></p1>
<p1><center> Electrolyte Species (5)</center></p1>
<p1><center> SEI Species (2)</center></p1>
<p1><center> Interface Species (1)</center></p1>



In [ ]:
sdot.reshape((sdot.size,1))

## Let's load something with more complex chemistry:

In [ ]:
infile = batfile_2
anode = ct.Solution(infile,anode_phase)
elyte = ct.Solution(infile,elyte_phase)
SEI = ct.Solution(infile,SEI_phase)
anode_elyte = ct.Interface(infile,anode_elyte_int,[anode,elyte,SEI])

## Explore the phase species:

In [ ]:
print(elyte.species_names)
print(SEI.species_names)

## Set the phase properties

In [ ]:
anode.TP = 300, ct.one_atm
elyte.TP = 300, ct.one_atm

anode.electric_potential = 1.2
anode_elyte.electric_potential = 1.2
elyte.electric_potential = 0.0

## Read out the production rates:

In [ ]:
sdot = anode_elyte.net_production_rates
sdot.reshape((sdot.size,1))

The order of phases is still the same:

<p1><center>Tungsten Species (?)</center></p1>
<p1><center> Electrolyte Species (?)</center></p1>
<p1><center> SEI Species (?)</center></p1>
<p1><center> Interface Species (?)</center></p1>

The phases just have different numbers of species.  However, we can use Cantera routines so that the derivative arrays are all correctly sized, automatically:

In [ ]:
print('n_anode: ',anode.n_species)
print('n_elyte: ',elyte.n_species)
print('n_SEI: ',SEI.n_species)
print('n_int: ',anode_elyte.n_species)

In [ ]:
V = np.linspace(1.5,0.25,15)
i = np.zeros_like(V)
for index,E in enumerate(V):
    anode.electric_potential = E
    i[index] = ct.faraday*anode_elyte.net_production_rates[0]
    print(round(E,2), round(i[index],2))

In [ ]:
plt.figure()
plt.plot(V,i)
plt.show()